In [1]:
import sys
import re
import glob
import codecs
import unicodedata
import os
import wave
import contextlib
import numpy as np
import pycrfsuite

In [124]:
'''f = codecs.open('train_pos.txt', 'r', encoding='utf-8')
p = codecs.open('train_normalized.txt', 'w', encoding='utf-8')
for line in f:
    line = unicodedata.normalize('NFC',line)
    line = re.sub(r"<[A-Z]{1,}\${0,}>", " ", line)
    line = re.sub(r"-", " ", line)
    line = re.sub(r"<", " ", line)
    line = re.sub(r">", " ", line)
    line = re.sub(r"#", " ", line)
    line = re.sub(r"/", " ", line)
    line = re.sub(r"\\", " ", line)
    line = re.sub(r",", " ", line)
    line = re.sub(u"،", " ", line)
    line = re.sub(u"؛", " ", line)
    line = re.sub(u"؟", " ", line)
    line = re.sub(r";", " ", line)
    line = re.sub(r"\"", " ", line)
    line = re.sub(u"۔", " ", line)
    line = re.sub(r"\.", " ", line)
    line = re.sub(r":", " ", line)
    line = re.sub(r"!", " ", line)
    line = re.sub(r"\'", " ", line)
    line = re.sub(r"%", u"٪", line)
    line = re.sub(u"ِ", "", line)
    line = re.sub(u"ُ", "", line)
    line = re.sub(u"َ", "", line)
    line = re.sub(u"ْ", "", line)
    line = re.sub(u"ٰ", "", line)
    line = re.sub(u"ً", "", line)
    line = re.sub(u"ّ", "", line)
    line = re.sub(u"\u200C{2,}", " ", line)
    line = re.sub(r"\s{2,}", " ", line)
    p.write(line.strip()+'\n')
f.close()
p.close()'''

In [129]:
'''f = codecs.open('train_normalized.txt', 'r', encoding='utf-8')
p = codecs.open('unique_characters.txt', 'w', encoding='utf-8')
characters = list()
for line in f:
    for letter in line:
        characters.append(letter)
f.close()
unique_characters = list(set(characters))
for i in unique_characters:
    p.write(i+'\n')
p.close()'''

In [2]:
sentences = list()
f = codecs.open('train_normalized.txt', 'r', encoding='utf-8')
for line in f:
    sentences.append(line)
f.close()

print len(sentences)
print sentences[100]

4325
منصوبے جو کہ مشتہرین کو اشتہاروں کے خرچ مستحکم رکھنے یا بڑھانے کے لیے بچت دیتے ہیں اب ہفت‌روزہ اخباروں کے لیے مستقل عمل بن گئے ہیں اور اس نے نیوزویک ٹائم وارنر انکارپوریٹڈ کے ٹائم میگزین اور مورٹیمر بی زکرمین کے یو ایس نیوز اور ورلڈ رپورٹ میں شدید مقابلے کو نمایاں کیا ہے



In [32]:
def prepare_sentence(sentence):
    sentence = re.sub(u"\s\u200C",u"\u200C", sentence)
    sentence = re.sub(u"\u200C\s",u"\u200C", sentence)
    
    lengths = [len(w) for w in sentence.split(" ")]
    positions = []

    next_pos = 0
    for length in lengths:
        next_pos = next_pos + length
        positions.append(next_pos)
    concatenated = sentence.replace(" ", "")

    chars = [c for c in concatenated]
    labels = [0 if not i in positions else 1 for i, c in enumerate(concatenated)]
    #Uncomment to incorporate ZWNJ
    '''for i, c in enumerate(chars):
        if c == u"\u200C" and i+1 < len(chars):
            labels[i+1] = 2
            del chars[i]
            del labels[i]'''
    
    return list(zip(chars, labels))

In [33]:
prepared_sentences = list()
for sentence in sentences:    
    prepared_sentences.append(prepare_sentence(sentence))
    
print([d for d in prepared_sentences[100]])

[(u'\u0645', 0), (u'\u0646', 0), (u'\u0635', 0), (u'\u0648', 0), (u'\u0628', 0), (u'\u06d2', 0), (u'\u062c', 1), (u'\u0648', 0), (u'\u06a9', 1), (u'\u06c1', 0), (u'\u0645', 1), (u'\u0634', 0), (u'\u062a', 0), (u'\u06c1', 0), (u'\u0631', 0), (u'\u06cc', 0), (u'\u0646', 0), (u'\u06a9', 1), (u'\u0648', 0), (u'\u0627', 1), (u'\u0634', 0), (u'\u062a', 0), (u'\u06c1', 0), (u'\u0627', 0), (u'\u0631', 0), (u'\u0648', 0), (u'\u06ba', 0), (u'\u06a9', 1), (u'\u06d2', 0), (u'\u062e', 1), (u'\u0631', 0), (u'\u0686', 0), (u'\u0645', 1), (u'\u0633', 0), (u'\u062a', 0), (u'\u062d', 0), (u'\u06a9', 0), (u'\u0645', 0), (u'\u0631', 1), (u'\u06a9', 0), (u'\u06be', 0), (u'\u0646', 0), (u'\u06d2', 0), (u'\u06cc', 1), (u'\u0627', 0), (u'\u0628', 1), (u'\u0691', 0), (u'\u06be', 0), (u'\u0627', 0), (u'\u0646', 0), (u'\u06d2', 0), (u'\u06a9', 1), (u'\u06d2', 0), (u'\u0644', 1), (u'\u06cc', 0), (u'\u06d2', 0), (u'\u0628', 1), (u'\u0686', 0), (u'\u062a', 0), (u'\u062f', 1), (u'\u06cc', 0), (u'\u062a', 0), (u'\u06

In [34]:
def create_char_features(sentence, i):
    features = [
        'bias',
        'char=' + sentence[i][0] 
    ]
    
    if i >= 1:
        features.extend([
            'char-1=' + sentence[i-1][0],
            'char-1:0=' + sentence[i-1][0] + sentence[i][0],
        ])
    else:
        features.append("BOS")
        
    if i >= 2:
        features.extend([
            'char-2=' + sentence[i-2][0],
            'char-2:0=' + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-2:-1=' + sentence[i-2][0] + sentence[i-1][0],
        ])
        
    if i >= 3:
        features.extend([
            'char-3:0=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-3:-1=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0],
        ])
        
        
    if i + 1 < len(sentence):
        features.extend([
            'char+1=' + sentence[i+1][0],
            'char:+1=' + sentence[i][0] + sentence[i+1][0],
        ])
    else:
        features.append("EOS")
        
    if i + 2 < len(sentence):
        features.extend([
            'char+2=' + sentence[i+2][0],
            'char:+2=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0],
            'char+1:+2=' + sentence[i+1][0] + sentence[i+2][0],
        ])
        
    if i + 3 < len(sentence):
        features.extend([
            'char:+3=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0]+ sentence[i+3][0],
            'char+1:+3=' + sentence[i+1][0] + sentence[i+2][0] + sentence[i+3][0],
        ])
    
    return features



def create_sentence_features(prepared_sentence):
    return [create_char_features(prepared_sentence, i) for i in range(len(prepared_sentence))]

def create_sentence_labels(prepared_sentence):
    return [str(part[1]) for part in prepared_sentence]

In [35]:
X_train = [create_sentence_features(ps) for ps in prepared_sentences[:-825]]
y_train = [create_sentence_labels(ps)   for ps in prepared_sentences[:-825]]

In [36]:
print X_train[0]
print y_train[0]

[['bias', u'char=\u0627', 'BOS', u'char+1=\u06a9', u'char:+1=\u0627\u06a9', u'char+2=\u0633', u'char:+2=\u0627\u06a9\u0633', u'char+1:+2=\u06a9\u0633', u'char:+3=\u0627\u06a9\u0633\u0679', u'char+1:+3=\u06a9\u0633\u0679'], ['bias', u'char=\u06a9', u'char-1=\u0627', u'char-1:0=\u0627\u06a9', u'char+1=\u0633', u'char:+1=\u06a9\u0633', u'char+2=\u0679', u'char:+2=\u06a9\u0633\u0679', u'char+1:+2=\u0633\u0679', u'char:+3=\u06a9\u0633\u0679\u06be', u'char+1:+3=\u0633\u0679\u06be'], ['bias', u'char=\u0633', u'char-1=\u06a9', u'char-1:0=\u06a9\u0633', u'char-2=\u0627', u'char-2:0=\u0627\u06a9\u0633', u'char-2:-1=\u0627\u06a9', u'char+1=\u0679', u'char:+1=\u0633\u0679', u'char+2=\u06be', u'char:+2=\u0633\u0679\u06be', u'char+1:+2=\u0679\u06be', u'char:+3=\u0633\u0679\u06be\u0633', u'char+1:+3=\u0679\u06be\u0633'], ['bias', u'char=\u0679', u'char-1=\u0633', u'char-1:0=\u0633\u0679', u'char-2=\u06a9', u'char-2:0=\u06a9\u0633\u0679', u'char-2:-1=\u06a9\u0633', u'char-3:0=\u0627\u06a9\u0633\u0679'

In [37]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [38]:
trainer.set_params({
    'c1': 1.0, 
    'c2': 1e-3,
    'max_iterations': 60,
    'feature.possible_transitions': True
})

In [39]:
trainer.train('urdu-text-segmentation.crfsuite')

In [40]:
tagger = pycrfsuite.Tagger()
tagger.open('urdu-text-segmentation.crfsuite')

In [41]:
def segment_sentence(sentence):
    sentence = sentence.replace(" ", "")
    #sentence = sentence.replace(u"\u200C", "") #Uncomment to incorporate ZWNJ
    prediction = tagger.tag(create_sentence_features(sentence))
    print prediction
    complete = ""
    for i, p in enumerate(prediction):
        if p == "1":
            complete += " " + sentence[i]
        #Uncomment to incorporate ZWNJ
        #elif p == "2":
        #    complete += u"\u200C" + sentence[i]
        else:
            complete += sentence[i]
    return complete

In [42]:
print(segment_sentence(u"شرح ‌فی‌صد تبدیلی سال کے آخر سے ہے"))

['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '0', '1', '0']
شرح‌فی‌صد تبدیلی سال کے آخر سے ہے


In [43]:
tp = 0.0
fp = 0.0
fn = 0.0
n_correct = 0.0
n_incorrect = 0.0

for s in prepared_sentences[-825:]:
    prediction = tagger.tag(create_sentence_features(s))
    correct = create_sentence_labels(s)
    zipped = list(zip(prediction, correct))
    tp +=        len([_ for l, c in zipped if l == c and l == "1"])
    fp +=        len([_ for l, c in zipped if l == "1" and c == "0"])
    fn +=        len([_ for l, c in zipped if l == "0" and c == "1"])
    n_incorrect += len([_ for l, c in zipped if l != c])
    n_correct   += len([_ for l, c in zipped if l == c])

In [44]:
print "Precision:", tp/(tp+fp)
print "Recall:", tp/(tp+fn)
print "Accuracy:", n_correct/(n_correct+n_incorrect)
print tp, fp, fn, n_correct, n_incorrect

Precision: 0.962643971053
Recall: 0.95878381808
Accuracy: 0.981084952437
18889.0 733.0 812.0 80136.0 1545.0
